# Blockchain for School Marks

This notebook demonstrates the creation of a simple blockchain to store school marks. Each student's data is represented as a block in the blockchain.

In [ ]:
# Imports
import os
import pandas as pd
import hashlib as hasher
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# Constants
CSV_FILE = "Marks.csv"

# Load and clean data
def load_marks(file_name):
    if not os.path.exists(file_name):
        raise FileNotFoundError(f"{file_name} not found.")
    
    df = pd.read_csv(file_name)
    df["Subject"] = df["Subject"].str.strip().replace({
        "math": "Mathematics", "Math": "Mathematics", "maths": "Mathematics",
        "sci": "Science", "Sci": "Science",
        "eng": "English", "Eng": "English"
    })
    required = {"Student", "Subject", "Marks", "Exam"}
    if not required.issubset(df.columns):
        raise ValueError(f"CSV must contain: {required}")
    
    return df.dropna(subset=["Marks"])

# Block class
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.compute_hash()

    def compute_hash(self):
        sha = hasher.sha256()
        sha.update(f"{self.index}{self.timestamp}{self.data}{self.previous_hash}".encode())
        return sha.hexdigest()

# Blockchain functions
def create_genesis_block():
    return Block(0, dt.datetime.now(), "Genesis Block", "0")

def add_block(last_block, data):
    return Block(last_block.index + 1, dt.datetime.now(), data, last_block.hash)

def build_blockchain(df):
    chain = [create_genesis_block()]
    last = chain[0]
    for _, row in df.iterrows():
        block = add_block(last, f"{row.to_dict()}")
        chain.append(block)
        last = block
    return chain

def blockchain_to_df(chain):
    return pd.DataFrame([{
        "Index": b.index,
        "Timestamp": b.timestamp,
        "Data": b.data,
        "Prev Hash": b.previous_hash,
        "Hash": b.hash
    } for b in chain])

def check_integrity(chain):
    print("\n🔐 Blockchain Integrity Check:")
    for i in range(1, len(chain)):
        if chain[i].previous_hash != chain[i-1].hash:
            print(f" Tampering detected at Block {i}")
            return False
    print(" Blockchain is valid.")
    return True

# Advanced search
def search_blockchain(chain, keyword):
    print(f"\n Searching for '{keyword}' in blockchain...")
    found = False
    for b in chain[1:]:
        if keyword.lower() in b.data.lower():
            print(f"Block #{b.index}: {b.data}")
            found = True
    if not found:
        print("No match found.")

# Performance Analysis
def add_performance_trends(df):
    avg = df.groupby("Student")["Marks"].mean().reset_index().rename(columns={"Marks": "Average_Marks"})
    df = df.merge(avg, on="Student")
    df["Trend"] = df.apply(lambda row: "Above Avg" if row["Marks"] > row["Average_Marks"]
        else "Below Avg" if row["Marks"] < row["Average_Marks"] else "At Avg", axis=1)
    return df

# Visuals
def plot_bar(df):
    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 5))
    sns.barplot(data=df, x="Student", y="Marks", hue="Subject", ci=None)
    plt.title("Student Performance by Subject")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def plot_pie(df):
    total = df.groupby("Subject")["Marks"].sum()
    plt.figure(figsize=(6, 6))
    total.plot.pie(autopct='%1.1f%%', startangle=90, colors=sns.color_palette("Set2"))
    plt.title("Total Marks by Subject")
    plt.ylabel("")
    plt.tight_layout()
    plt.show()

# CLI-style interface
def main():
    df = load_marks(CSV_FILE)
    chain = build_blockchain(df)
    blockchain_df = blockchain_to_df(chain)
    trends_df = add_performance_trends(df)

    print("📘 Blockchain Overview:")
    display(blockchain_df)
    print("\n📊 Marks Overview:")
    print("\n📊 Student Performance vs Average:")
    display(trends_df.sort_values(["Student", "Exam"]))
    print("\n📊 Blockchain Data:")
    plot_bar(trends_df)
    plot_pie(trends_df)
    
    check_integrity(chain)

    # Search options
    keyword = input("\n🔍 Enter keyword to search (e.g., student name, subject): ").strip()
    if keyword:
        search_blockchain(chain, keyword)

# Run everything
main()
